# Obtain SOM

In [14]:
from django.test import TestCase
import json
import cv2
import numpy as np
import time
import os 
import torch
from SOM.UiComponent import UiComponent
from SOM import utils
from SOM import ip_draw 
import random
from SOM.sam import get_sam_gui_components_crops

In [15]:
def obtain_som(root_path, screenshots_names, image_folder_path, sam_checkpoint):
    COMPOS_JSON_PATH = root_path+'compos_json/'
    MASK_JSON_PATH = root_path+'mask_json/'
    TIME_JSON_PATH = root_path+'time_json/'

    for i,name in enumerate(screenshots_names):
        name = name.split('.')[0]
        recortes_path=root_path+'compos_npy/'+name+'.npy'
        path_to_save_mask_npy=root_path+'mask_elements_npy/'+name
        checkpoint_path="../../checkpoints/" # Suposed to be: featureextraction/SOM/checkpoints
        clips, uicompos, mask_json, compos_json, arrays_dict,dict_times = get_sam_gui_components_crops(param_img_root=image_folder_path, path_to_save_bordered_images=root_path, image_names=screenshots_names, img_index=i, checkpoint_path=checkpoint_path, checkpoint=sam_checkpoint)

        time0 = time.time()
        with open(COMPOS_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(compos_json)

        with open(MASK_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(mask_json)

        compos_aux = np.array(clips)
        np.save(recortes_path,compos_aux)

        # npy for each segmentation of SAM
        path=path_to_save_mask_npy
        for n in ['segmentation','crop_box']:
            path_element = path+'_'+n+'_'+sam_checkpoint+'.npy'
            aux = np.array(arrays_dict[n])
            np.save(path_element,aux)
        time1=time.time()
        
        dict_times['saving_results']=time1-time0

        times_json = json.dumps(dict_times)
        with open(TIME_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(times_json)

        print('There are {} components'.format(len(uicompos)))

In [16]:
screenshots_names = [
    "screenshot0012.JPEG",
    "screenshot0014.JPEG",
    "screenshot0015.JPEG",
    "screenshot0016.JPEG",
    "screenshot0017.JPEG",
    "screenshot0018.JPEG",
    "screenshot0019.JPEG",
    "screenshot0020.JPEG",
    "screenshot0022.JPEG",
    "screenshot0023.JPEG",
    "screenshot0024.JPEG",
    "screenshot0025.JPEG",
    "screenshot0026.JPEG"
]

root = "../../../resources/gaze/gaze4/"
obtain_som(root, screenshots_names, root, "l")

../../resources/gaze/gaze4/ip
True


C:\Users\pablo\AppData\Local\Temp\ipykernel_16520\413371598.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  compos_aux = np.array(clips)


There are 107 components
../../resources/gaze/gaze4/ip
True
There are 112 components
../../resources/gaze/gaze4/ip
True
There are 64 components
../../resources/gaze/gaze4/ip
True
There are 79 components
../../resources/gaze/gaze4/ip
True
There are 91 components
../../resources/gaze/gaze4/ip
True
There are 85 components
../../resources/gaze/gaze4/ip
True
There are 111 components
../../resources/gaze/gaze4/ip
True
There are 104 components
../../resources/gaze/gaze4/ip
True
There are 110 components
../../resources/gaze/gaze4/ip
True
There are 64 components
../../resources/gaze/gaze4/ip
True
There are 70 components
../../resources/gaze/gaze4/ip
True
There are 85 components
../../resources/gaze/gaze4/ip
True
There are 71 components


# Metadata to Dataframe

In [4]:
import json
import pandas as pd

root_path = "/screenrpa/resources/metadata/"

json_filenames = ["221-metainfo.json", "222-metainfo.json"]  

json_files = [root_path + filename for filename in json_filenames]

for i, json_path in enumerate(json_files):
    json_file = json.load(open(json_path))

    for key in json_file:
        subjson = json_file[key]["ui_elements_detection"]["screenshots"]
        
        columns_names = [
            "detect_images_components duration",
            "detect_images_components #UICompos",
            "detect_images_components get_binary_map_time",
            "detect_images_components get_component_detection",
            "detect_images_components merge_intersected_compos",
            "detect_images_components compo_block_recognition",
            "detect_images_components nesting_inspection",
            "detect_images_components save_detection_result",
            "detect_images_components compo_clipping"
            ]

        df = pd.DataFrame([], columns=["screenshot"] + columns_names)

        for j, screen_key in enumerate(subjson):
            row = [screen_key]
            for column_name in columns_names:
                row += [subjson[screen_key][column_name]]
            df.loc[j] = row
            
        df.to_csv(root_path + json_filenames[i] + "_metrics.csv")
